# **🏠 지하철 좌표 및 유동인구**
> 아파트와 버스 정류장 거리를 계산해서 150m 이내 정류장 개수를 count
> 지하철 데이터에 버스 데이터 열 붙여넣기

## Contents
- Library Import
- Data Load
- Haversin Distance Calculate

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance
import time

## 2. Data Load

#### 2.1. 데이터 로드

In [2]:
# 데이터 불러오기
address = pd.read_csv('../data/지번주소별_유동인구_역세권_역까지거리.csv')
bus = pd.read_csv('../data/bus_feature.csv')

In [3]:
# 위경도 좌표
bus2 = bus[['X좌표', 'Y좌표']]

In [4]:
# 본격적 하버사인 거리 구하기
veryhard = pd.merge(address, bus2, how='cross')

In [5]:
# 하버사인 거리
lat1 = np.deg2rad(veryhard['X좌표'])
lat2 = np.deg2rad(veryhard['좌표X뉴'])
lon1 = np.deg2rad(veryhard['Y좌표'])
lon2 = np.deg2rad(veryhard['좌표Y뉴'])
dlat = lat2 - lat1
dlon = lon1 - lon2
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [6]:
# 미터단위 거리 구하기
veryhard['정류장까지_거리'] = c * 6371 * 1000

In [7]:
# 거리자르기
min_value = veryhard.loc[veryhard.groupby('지번주소')['정류장까지_거리'].idxmin()]

In [44]:
# 일단 저장해 저장 필수로해
min_value.to_csv('../data/지번주소별_유동인구_역세권_역까지거리_정류장까지거리.csv', index=False)